In [23]:
import pathlib
import glob
import os
import sys
import json
import seaborn as sns

from matplotlib import pyplot as plt

from analysis import Run, RunAnalyser, CriterionPlotter, load_all_runs_from_a_dir

In [24]:
COLORS = []
for idx, c in enumerate(sns.color_palette()):
    print(f"\\definecolor{{color{idx}}}{{RGB}}{{{','.join(str(int(k * 255)) for k in c)}}}")
    COLORS.append(f'color{idx}')

\definecolor{color0}{RGB}{31,119,180}
\definecolor{color1}{RGB}{255,127,14}
\definecolor{color2}{RGB}{44,160,44}
\definecolor{color3}{RGB}{214,39,40}
\definecolor{color4}{RGB}{148,103,189}
\definecolor{color5}{RGB}{140,86,75}
\definecolor{color6}{RGB}{227,119,194}
\definecolor{color7}{RGB}{127,127,127}
\definecolor{color8}{RGB}{188,189,34}
\definecolor{color9}{RGB}{23,190,207}


In [25]:
CITY_CATEGORIES = ['road_construction_site', 'crowd', 'large_trash_pile', 'fire', 'car']
FOREST_CATEGORIES = ['campsite', 'trash_pile', 'person', 'forest_fire', 'building']
ALL_CATEGORIES = ['large_trash_pile', 'fire', 'car', 'trash_pile', 'person', 'building','campsite', 'forest_fire', 'road_construction_site', 'crowd']
MODELS = [
    'GPT4o',
    'Sonnet',
    'Gemini',
    'Phi',
    'InternVL',
    'llava-interleave-7b',
    'Qwen2-VL-72B',
    'llava-onevision',
    'Pixtral-Large-Instruct-2411',
]
colors_map = {k: v for k, v in zip(MODELS, COLORS)}

In [26]:
colors_map

{'GPT4o': 'color0',
 'Sonnet': 'color1',
 'Gemini': 'color2',
 'Phi': 'color3',
 'InternVL': 'color4',
 'llava-interleave-7b': 'color5',
 'Qwen2-VL-72B': 'color6',
 'llava-onevision': 'color7',
 'Pixtral-Large-Instruct-2411': 'color8'}

In [27]:
# Can be used for forest as well
def class_aggregation_function(run: Run):
    object_type = str(run.object_type).lower()

    if "car" in object_type:
        return "car"
    elif "pickup" in object_type:
        return "car"
    elif "truck" in object_type:
        return "car"
    return object_type

def anomaly_aggregation_function(run: Run):
    object_type = str(run.object_type).lower()

    if "anomaly" in object_type:
        return "anomaly"
    return 'main'

def success_criterion(run):
    return run.model_claimed and RunAnalyser(run).maciek_criterion_satisfied(10)

## Settings

In [35]:
base_path = "../all_logs"
suffix = '-CityNew'
all_exp = [(m, os.path.join(base_path, m + suffix)) for m in MODELS]
print(all_exp)
categories = ALL_CATEGORIES
agg_fn = anomaly_aggregation_function
# agg_fn = class_aggregation_function
add_forest = True

[('GPT4o', '../all_logs/GPT4o-CityNew'), ('Sonnet', '../all_logs/Sonnet-CityNew'), ('Gemini', '../all_logs/Gemini-CityNew'), ('Phi', '../all_logs/Phi-CityNew'), ('InternVL', '../all_logs/InternVL-CityNew'), ('llava-interleave-7b', '../all_logs/llava-interleave-7b-CityNew'), ('Qwen2-VL-72B', '../all_logs/Qwen2-VL-72B-CityNew'), ('llava-onevision', '../all_logs/llava-onevision-CityNew'), ('Pixtral-Large-Instruct-2411', '../all_logs/Pixtral-Large-Instruct-2411-CityNew')]


In [36]:
plots = []
for model_name, exp_path in all_exp:
    runs = load_all_runs_from_a_dir(pathlib.Path(exp_path))
    if add_forest:
        runs += load_all_runs_from_a_dir(pathlib.Path(exp_path.replace('City', 'Forest')))
    plotter = CriterionPlotter(runs)
    runs_aggregated_per_type = plotter.aggregate_runs_per_function(agg_fn)
    claimed_stats = plotter.plot_accuracy_in_aggregated_runs(runs_aggregated_per_type, None, success_criterion=success_criterion)
    unclaimed_stats = plotter.plot_accuracy_in_aggregated_runs(runs_aggregated_per_type, None,
                                                           success_criterion=lambda x: RunAnalyser(
                                                               x).maciek_criterion_satisfied(10))
    plots.append((model_name, claimed_stats, unclaimed_stats))

## Radar plot

In [34]:
for name, stat, _ in plots:
    print(f'\\tkzKiviatLine[thick,color={colors_map[name]}](', end='')
    print(','.join([f"{stat[c]['mean'] * 100:.3f}" if c in stat else '' for c in categories]), end='')
    print(')')

\tkzKiviatLine[thick,color=color0](19.570,41.300,31.110,43.400,19.050,51.110,68.750,53.570,30.000,56.520)
\tkzKiviatLine[thick,color=color1](23.910,23.910,37.780,37.740,40.480,68.890,59.380,60.710,17.500,65.220)
\tkzKiviatLine[thick,color=color2](39.130,43.480,42.220,41.510,23.810,48.890,40.620,64.290,30.000,60.870)
\tkzKiviatLine[thick,color=color3](0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000)
\tkzKiviatLine[thick,color=color4](0.000,2.170,0.000,3.770,2.380,4.440,0.000,7.140,0.000,8.700)
\tkzKiviatLine[thick,color=color5](0.000,6.520,0.000,0.000,0.000,0.000,0.000,10.710,0.000,0.000)
\tkzKiviatLine[thick,color=color6](13.040,21.740,17.780,9.430,4.760,28.890,6.250,35.710,12.500,30.430)
\tkzKiviatLine[thick,color=color7](2.170,30.430,4.440,9.430,9.520,15.560,25.000,42.860,12.500,4.350)
\tkzKiviatLine[thick,color=color8](21.740,21.740,20.000,26.420,16.670,62.220,43.750,46.430,10.000,43.480)


In [8]:
for k, v in colors_map.items():
    print(f'\\addlegendimage{{{v},ultra thick}}')
    print(f'\\addlegendentry{{{k}}};')
    

\addlegendimage{color0,ultra thick}
\addlegendentry{GPT4o};
\addlegendimage{color1,ultra thick}
\addlegendentry{Sonnet};
\addlegendimage{color2,ultra thick}
\addlegendentry{Gemini};
\addlegendimage{color3,ultra thick}
\addlegendentry{Phi};
\addlegendimage{color4,ultra thick}
\addlegendentry{InternVL};
\addlegendimage{color5,ultra thick}
\addlegendentry{llava-interleave-7b};
\addlegendimage{color6,ultra thick}
\addlegendentry{Qwen2-VL-72B};
\addlegendimage{color7,ultra thick}
\addlegendentry{llava-onevision};
\addlegendimage{color8,ultra thick}
\addlegendentry{Pixtral-Large-Instruct-2411};


## Main plot

In [174]:
for idx, (m, p, _) in enumerate(plots):
    p = p['main']
    mean = p['mean']
    lower = p['mean'] - p['conf_int'][0]
    upper = p['conf_int'][1] - p['mean']
    print(f'\\addplot[style={{fill={colors_map[m]}}},error bars/.cd, y dir=both, y explicit] coordinates {{({idx+1}, {p["mean"]*100:.3f}) += (0,{upper*100:.3f}) -= (0,{lower*100:.3f})}};')



\addplot[style={fill=color0},error bars/.cd, y dir=both, y explicit] coordinates {(1, 45.500) += (0,7.170) -= (0,7.040)};
\addplot[style={fill=color1},error bars/.cd, y dir=both, y explicit] coordinates {(2, 52.000) += (0,7.100) -= (0,7.160)};
\addplot[style={fill=color2},error bars/.cd, y dir=both, y explicit] coordinates {(3, 42.500) += (0,7.170) -= (0,6.940)};
\addplot[style={fill=color3},error bars/.cd, y dir=both, y explicit] coordinates {(4, 0.000) += (0,1.830) -= (0,0.000)};
\addplot[style={fill=color4},error bars/.cd, y dir=both, y explicit] coordinates {(5, 3.500) += (0,3.580) -= (0,2.080)};
\addplot[style={fill=color5},error bars/.cd, y dir=both, y explicit] coordinates {(6, 1.500) += (0,2.820) -= (0,1.190)};
\addplot[style={fill=color6},error bars/.cd, y dir=both, y explicit] coordinates {(7, 16.000) += (0,5.830) -= (0,4.790)};
\addplot[style={fill=color7},error bars/.cd, y dir=both, y explicit] coordinates {(8, 18.000) += (0,6.040) -= (0,5.060)};
\addplot[style={fill=color8

## Anomaly

In [13]:
for idx, (m, p, _) in enumerate(plots):
    p = p['anomaly']
    mean = p['mean']
    lower = p['mean'] - p['conf_int'][0]
    upper = p['conf_int'][1] - p['mean']
    print(f'\\addplot[style={{fill={colors_map[m]}}},error bars/.cd, y dir=both, y explicit] coordinates {{({idx+1}, {p["mean"]*100:.3f}) += (0,{upper*100:.3f}) -= (0,{lower*100:.3f})}};')



\addplot[style={fill=color0},error bars/.cd, y dir=both, y explicit] coordinates {(1, 15.000) += (0,8.530) -= (0,6.350)};
\addplot[style={fill=color1},error bars/.cd, y dir=both, y explicit] coordinates {(2, 18.000) += (0,8.950) -= (0,6.970)};
\addplot[style={fill=color2},error bars/.cd, y dir=both, y explicit] coordinates {(3, 25.000) += (0,9.660) -= (0,8.120)};
\addplot[style={fill=color3},error bars/.cd, y dir=both, y explicit] coordinates {(4, 0.000) += (0,3.620) -= (0,0.000)};
\addplot[style={fill=color4},error bars/.cd, y dir=both, y explicit] coordinates {(5, 1.000) += (0,4.450) -= (0,0.970)};
\addplot[style={fill=color5},error bars/.cd, y dir=both, y explicit] coordinates {(6, 0.000) += (0,3.620) -= (0,0.000)};
\addplot[style={fill=color6},error bars/.cd, y dir=both, y explicit] coordinates {(7, 5.000) += (0,6.280) -= (0,3.360)};
\addplot[style={fill=color7},error bars/.cd, y dir=both, y explicit] coordinates {(8, 6.000) += (0,6.600) -= (0,3.770)};
\addplot[style={fill=color8},

## Non-claim

In [44]:
print(r'\addplot+[ybar] plot coordinates {', end='')
for idx, (m, p, pn) in enumerate(plots):
    p = p['main']
    pn = pn['main']   
    print(f'({idx+1},{p["mean"]*100:.3f})', end='')
print('};')
print(r'\addplot+[ybar] plot coordinates {', end='')
for idx, (m, p, pn) in enumerate(plots):
    p = p['main']
    pn = pn['main']   
    print(f'({idx+1},{(pn["mean"] - p["mean"])*100:.3f})', end='')
print('};')



    

\addplot+[ybar] plot coordinates {(1,39.500)(2,41.250)(3,42.000)(4,0.000)(5,2.500)(6,1.500)(7,17.000)(8,14.750)(9,29.750)};
\addplot+[ybar] plot coordinates {(1,0.250)(2,0.250)(3,1.500)(4,4.250)(5,4.750)(6,7.750)(7,3.250)(8,1.500)(9,0.250)};
